In [14]:
150+126
#gransger causality

276

In [1]:
import torch
from tqdm import tqdm
import torch.nn as nn

from load_data import read_data_arrays, data_file_names, standardize_data, data_loader
from models import ChronoNet
from utils import cal_accuracy, evaluate_model 


BATCH_SIZE = 128
#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
DEVICE = torch.device("cpu")
NUM_EPOCHS = 5

print("Reading Data....")
sample_size = 500
data_files = data_file_names(sample_size)
(train_features, val_features, test_features,
 train_labels, val_labels, test_labels, test_lengths) = read_data_arrays(
    data_files)
    
print("Scaling Data....")
train_features, val_features, test_features = standardize_data(
    train_features, val_features, test_features)
    
print("Data Loader....")
train_iter = data_loader(train_features, train_labels, DEVICE, BATCH_SIZE)
val_iter = data_loader(val_features, val_labels, DEVICE, BATCH_SIZE)
test_iter = data_loader(test_features, test_labels, DEVICE, BATCH_SIZE, shuffle=False)
    
print("Training Model....")
n_chans = 19
model=ChronoNet(n_chans)
model.to(DEVICE)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, NUM_EPOCHS + 1):
    print("Epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1), "Accuracy: ", 
        cal_accuracy(model, train_iter)[0])
    print("Val loss:", val_loss, ", Accuracy: ", 
        cal_accuracy(model, val_iter)[0])

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Reading Data....
Scaling Data....
Data Loader....
Training Model....
Epoch 1


100%|█████████████████████████████████████████| 692/692 [04:07<00:00,  2.79it/s]


Train loss: 0.6070634114604465 Accuracy:  0.8287796610169491
Val loss: 0.5940634066408331 , Accuracy:  0.8055932203389831
Epoch 2


100%|█████████████████████████████████████████| 692/692 [03:51<00:00,  2.99it/s]


...

Train loss: 0.5733702255983573 Accuracy:  0.8348248587570621
Val loss: 0.5903343382851902 , Accuracy:  0.7971186440677966
Epoch 4


100%|█████████████████████████████████████████| 692/692 [03:35<00:00,  3.21it/s]


Train loss: 0.5674663751972893 Accuracy:  0.8570621468926554
Val loss: 0.599976393825564 , Accuracy:  0.8138983050847458
Epoch 5


100%|█████████████████████████████████████████| 692/692 [03:36<00:00,  3.19it/s]


Train loss: 0.5658425752482662 Accuracy:  0.86
Val loss: 0.5895577963296469 , Accuracy:  0.8092203389830509


In [2]:
print("Accuracy: ", cal_accuracy(model, test_iter)[0])
print("Confusion Matrix: ", cal_accuracy(model, test_iter)[1])

Accuracy:  0.7954126750184229
Confusion Matrix:  [[15516  2184]
 [ 4479 10389]]


In [3]:
ytrue = []
ypreds = []
with torch.no_grad():
    for x, y in test_iter:
        yhat = model(x)
        yhat = [0 if i<0.5 else 1 for i in yhat]
        ytrue.extend(list(y.numpy()))
        ypreds.extend(yhat)

In [47]:
from collections import Counter

y_final = []
yhat_final = []
for i in range(0, len(ytrue), 118): 
    major_y_final = Counter(ytrue[i: i+118]).most_common(1)[0][0]
    major_yhat_final = Counter(ypreds[i: i+118]).most_common(1)[0][0]
    y_final.append(major_y_final)
    yhat_final.append(major_yhat_final)

In [48]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(y_final, yhat_final), confusion_matrix(y_final, yhat_final)

(0.822463768115942,
 array([[138,  12],
        [ 37,  89]]))